In [1]:
import numpy as np
import scipy.linalg as la
import hamiltonian as hm
%matplotlib inline
import matplotlib.pyplot as plt

In [59]:
L     = 11
vs    = np.asarray([1,2,3,4,5])
sites = np.arange(L)

In [60]:
times = []
for v in vs: times.extend(sites/v)

In [67]:
times = list(dict.fromkeys(times))
times.sort()
print(times)

[0.0, 0.20000000000000001, 0.25, 0.33333333333333331, 0.40000000000000002, 0.5, 0.59999999999999998, 0.66666666666666663, 0.75, 0.80000000000000004, 1.0, 1.2, 1.25, 1.3333333333333333, 1.3999999999999999, 1.5, 1.6000000000000001, 1.6666666666666667, 1.75, 1.8, 2.0, 2.25, 2.3333333333333335, 2.5, 2.6666666666666665, 3.0, 3.3333333333333335, 3.5, 4.0, 4.5, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]


In [71]:
sites_at_ts = []
for time in times:
    sites_at_t = []
    for v in vs: 
        site = (int) (np.round(time*v))
        if np.isclose((time*v), site) and (site < L) and not site in sites_at_t: 
            sites_at_t.append(site)
    sites_at_ts.append(sites_at_t)

In [74]:
for idx, val in enumerate(times):
    print(val, sites_at_ts[idx])

0.0 [0]
0.2 [1]
0.25 [1]
0.333333333333 [1]
0.4 [2]
0.5 [1, 2]
0.6 [3]
0.666666666667 [2]
0.75 [3]
0.8 [4]
1.0 [1, 2, 3, 4, 5]
1.2 [6]
1.25 [5]
1.33333333333 [4]
1.4 [7]
1.5 [3, 6]
1.6 [8]
1.66666666667 [5]
1.75 [7]
1.8 [9]
2.0 [2, 4, 6, 8, 10]
2.25 [9]
2.33333333333 [7]
2.5 [5, 10]
2.66666666667 [8]
3.0 [3, 6, 9]
3.33333333333 [10]
3.5 [7]
4.0 [4, 8]
4.5 [9]
5.0 [5, 10]
6.0 [6]
7.0 [7]
8.0 [8]
9.0 [9]
10.0 [10]
